<a href="https://colab.research.google.com/github/SDS-AAU/M1-2019/blob/master/notebooks/SDS_M1_S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installing some libraries that we are goint to use later
# seaborn (right version for plotting), hdbscan for clustering and rgeocoder for reverse geocoding offline
!pip3 install seaborn==0.9.0 hdbscan rgeocoder

# A quick dive into nomad geography and the gig-economy - Unsupervised ML
### Roman Jurowetzki - 10/9 - 2019; roman@business.aau.dk

In this tutorial you will learn how to work with dimensionality reduction and clustering techniques. We will have a look at data that comes in slightly different shapes and how we can aggregate it to identify latent patterns in a population.

We will use a dataset on cities worldwide (same source as the 1. miniassignment)to get familiar with dimensionality reduction.

Then, we will use a dataset of jobs performed by 1000 freelance-workers.

![](https://www.phocuswire.com/uploadedimages/uploads/2013/04/global-routes.jpg?width=800&height=400&scale=both&mode=crop)

In [0]:
#Pandas handles tabular data
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x) # turn off scientific notation and too much decimal blah
import seaborn as sns
sns.set_style("darkgrid")

import matplotlib.pyplot as plt # plotting library

#Numpy for linear algebra & co
import numpy as np


In [0]:
# We open the data directly from Github
cities = pd.read_csv('https://github.com/SDS-AAU/M1-2019/raw/master/data/nomad_cities.csv', sep='\t')

The data used in this tutorial is taken with permission form https://nomadlist.com/

In [0]:
# Let's have a quick look at the imported data
cities.info()

For some reason (data is messy) we have the name of the city as well as longitude/langitude but we don't have the country or region names.
Let's try to fix that.

One approach could be based on the lon/lat values, as city names may non-unique.

For this we need to start by "looking up" the country name for each specific coordinate

We can use an offline reverse geocoder package for that.

In [0]:
#Load and instantiate the reversegeocoder

from rgeocoder import ReverseGeocoder
rg = ReverseGeocoder()

In [0]:
# That's how it works

location = rg.nearest(56.457, 10.039) #lat, lon

In [0]:
# Let's see what we get
print(location.name)
print(location.admin1)
print(location.admin2)
print(location.cc)

In [0]:
# A smart way to get all geocoding done in one line

cities['countrycode'] = cities.apply(lambda t: rg.nearest(t['latitude'],t['longitude']).cc, axis=1)

in the above cell we apply the 


```
rg.nearest(t['latitude'],t['longitude']).cc
```

function to each row t of the dataframe cities


```
lambda t:
```

is a rather scary syntactic way of calling an anonymous function

Basically, we could say:


*   For each row of the cities dataframe
*   take the value of the longitude and latitude columns
*   look up the corresponding place
*   write the country code "cc" into a new column "alpha-2"

Now that we have exact country codes for each city, we need to find the regions these countries belong to

Luckily it's easy to find tables listing this information on the internet.




In [0]:
# Download a recent country info table
c = pd.read_csv('https://github.com/SDS-AAU/M1-2019/raw/master/data/countrylist.csv')

In [0]:
# Quick check
c.head()

In [0]:
 # Merge this lookup table with our initial cities list
 cities = cities.merge(c, left_on='countrycode', right_on='alpha_2')

In [0]:
# Let's select interesting variables for the analysis

vars_analysis = ["cost_nomad", "cost_coworking", "cost_expat", "coffee_in_cafe", "cost_beer", # costs
          "places_to_work", "free_wifi_available", "internet_speed", # work
          "freedom_score", "peace_score", "safety", "fragile_states_index", "press_freedom_index", # safety & freedom
          "female_friendly", "lgbt_friendly", "friendly_to_foreigners", "racism", # friendly
          "leisure","life_score","nightlife","weed"] # fun 

vars_descr = ["nomad_score", "cost_nomad", "places_to_work", "freedom_score", "friendly_to_foreigners", "life_score"]

In [0]:
# And use the selection to only extract these variables into a new object

data = cities[vars_analysis]

descr = cities[vars_descr]

In [0]:
# Quick check

data.info()

We can see some strange things going with the data types. Some are floats or ints as we would expect but others are objects

In [0]:
print(data['peace_score'].unique())
print(data['freedom_score'].unique())
print(data['fragile_states_index'].unique())
print(data['press_freedom_index'].unique())

This is a problem that probably has something to do with the scraping process. Let's try to turn everything into floats and if we can't we just put in a missing value statement

In [0]:
# This function will fo exactly that
def floater(x):
  try: #Try to
    return float(x) #Turn X into a floating point number
  except ValueError: #In case a ValueError occurs
    return np.nan #Turn X into np.nan (missing value placeholder)

In [0]:
# map applies the defined function to every observation

data.loc[:,'peace_score'] = data['peace_score'].map(floater)
data.loc[:,'freedom_score'] = data['freedom_score'].map(floater)
data.loc[:,'fragile_states_index'] = data['fragile_states_index'].map(floater)
data.loc[:,'press_freedom_index'] = data['press_freedom_index'].map(floater)

In [0]:
# yup, some missing data...
data.isnull().sum()

Now we are facing a new problem - Missing Data. While it would be easier to just kick out these cities, we will try to do better and impute.

Read up on imputation in the documentation of the package: https://github.com/iskandr/fancyimpute
also here: https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87

There are many other packages and modules that perform imputation. I found facyimpute the least problematic so far


In [0]:
# Import the imputation package
from fancyimpute import SoftImpute, SimpleFill

In [0]:
# Easy version: Just replace missing values by the mean of the column
data_imp = SimpleFill(fill_method='mean').fit_transform(data)

In [0]:
# We can have a quick look
pd.DataFrame(data_imp, columns=data.columns).head()

In [0]:
# Or, we go crazy and use a neural network powered method
data_imp = SoftImpute().fit_transform(data)

Before doing any kind of machine learning, especially when using neural networks and similar, it is important to normalise the data. One good way of doing it is using standardisation.
From each value we substract the sample mean and divide by the standard deviation.

In [0]:
# Let's standard-scale our data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_scaled = scaler.fit_transform(data_imp)

As you can see, now our data has a mean of 0 and a standard deviation of one.

In [0]:
pd.DataFrame(data_scaled, columns=data.columns).describe()

Let's try out dimensionality reduction with Principle Component Analysis: PCA
Check out this for details on PCA: https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html


Want to know more about dimensionality reduction?
https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/


In [0]:
# Import the module and instantiate a PCA object
from sklearn.decomposition import PCA
pca = PCA(n_components=7) #We pick 7 as the number of components...just because (it's a 3rd of the columns available)

# Fit and transform the data
data_reduced = pca.fit_transform(data_scaled)

In [0]:
# Make sure the data shape is as it should be
data_reduced.shape

In [0]:
plot_data = pd.DataFrame({'evr': pca.explained_variance_ratio_, 'cumsum_evr': np.cumsum(pca.explained_variance_ratio_)}).stack()

In [0]:
# Is 7 components really a good choice?
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(y = plot_data.values, x = plot_data.index.get_level_values(0), hue=plot_data.index.get_level_values(1))


In [0]:
# How mach "information" do we kick out?
pca.explained_variance_ratio_.sum()

Reducing the dimensionality is a nice way to make the data more approachable. In the present case, 21 variables is not a large number. However, the initial number of variables may be much higher (as we will see in the next dataset)

Now we can cluster the data using the most common and simple K-means algorithm.
https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html


### How many clusters?

Elbow - method

In [0]:
from sklearn.cluster import KMeans

In [0]:
inertia = []
for i in range(1,11):
  k_means = KMeans(n_clusters=i)
  inertia.append(k_means.fit(data_reduced).inertia_)

sns.lineplot(y = inertia, x = range(1,11))

In [0]:
# Let's cluster our data

clusterer = KMeans(n_clusters=3)
clusterer.fit(data_reduced)

In [0]:
# Now we can plot in our points with some coloring

plt.figure(figsize=(12,12))
g = sns.scatterplot(data_reduced[:,0], data_reduced[:,1], hue=clusterer.labels_,
               legend='full', palette='viridis')

legend = g.get_legend()

We can certainly make things much more fancy. However, as you can see, that will require more work.

We can use Bokeh for an interactive visualisation.

In [0]:
# Load the needed bokeh modules

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap

In [0]:
# Define the data that we are going to use as a dictionary

d = {'y':data_reduced[:,1],'x':data_reduced[:,0], 'place': cities.place, 
     'cluster': pd.Series(clusterer.labels_).map({0:'a',1:'b',2:'c'}),
     'country':cities['alpha_2'],
     'region':cities['sub_region']}

In [0]:
# Defineand transform a color-palette

colors = factor_cmap('cluster', palette=Spectral6, factors=d['cluster'].unique())

In [0]:
# Transform the data to Bokeh format
d = ColumnDataSource(d)

In [0]:
# Define interactive tooling and plot for notebook output

output_notebook()

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"
p = figure(tools=TOOLS)
p.hover.tooltips = [('Place', "@place"),('Country', "@country"),('Region', "@region")]
p.scatter(x='x', y='y',fill_alpha=0.8,
          color = colors,
          line_color = None,
          radius = 0.1,
          source=d)
show(p)

In [0]:
# Let's write our cluster numbers into the initial "real data"

descr['cluster'] = clusterer.labels_
cities['cluster'] = clusterer.labels_

In [0]:
# Now we can try out some basic descriptive statistics

descr.groupby('cluster').mean()

In [0]:
# Which cluster ranks lowest when it comes to the nomad score? And what are the cities in this cluster?

cities[cities.cluster == 2].sort_values('nomad_score', ascending=True)['place'][:10]

## Gig portfolios of online freelancers

![alt text](http://sds-datacrunch.aau.dk/public/adult-alone-bar-1308625.jpg)

You get gig-portfolio data for 1000 online freelancers – overall ~35k gigs. Given this data can you identify professional patterns?

This part of the tutorial is a short version based on the analysis performed for this paper:
**Career Paths in Digital Marketplaces: Same, same but different?**
With Mareike Seifried and Tobias Kretschmer
LMU Munich
Munich School of Management

 https://conference.druid.dk/acc_papers/48ox0g0vwmp0vvx8gj7lzwhbimflf0.pdf


In [0]:
# loading the data?
data = pd.read_csv('http://sds-datacrunch.aau.dk/public/feelance_eda.csv')

In [0]:
# Quick data exploration
data.info()

In [0]:
data.head()

In [0]:
# How does one portfolio look like?
data[data.f_id == 78].sub_category

#### Assembling gig-portfolios from a gig-list

A bit some thing on loops and slightly more advanced  stuff.

the next couple of cells will show how you can 
- add stuff to lists and how to 
- work with loops

In [0]:
empty_list = []

print(empty_list)

In [0]:
empty_list.append(1)

print(empty_list)

In [0]:
empty_list.append("i don't want to be in that list")

print(empty_list)

In [0]:
empty_list.append(['🙉','👽','🐼'])

print(empty_list)

In [0]:
# Let's get the panda out! --> '🐼'
empty_list[2][2]

In [0]:
empty_list.extend(['🐧','🍅','🤘'])

print(empty_list)

In [0]:
# LOOPS

emoji_list = ['🙉', '👽', '🐼','🐧','🍅','🤘']


for some_emoji in emoji_list:
  print(50 * some_emoji)

In [0]:
# LIST COMPREHENSIONS (mini-loops in Python)

[50*x for x in emoji_list]

From here we can try to assemble job-portfolios by making a list of lists with performed gigs for individual workers

In [0]:
# individual freelancers
workers = data.f_id.unique()

In [0]:
#create empty list
stuff_people_do = []

for some_worker_id in workers: #initiate loop
  stuff = list(data[data.f_id == some_worker_id].sub_category) # extract portfolio for a single worker
  stuff_people_do.append((some_worker_id, stuff)) # append portfolio to the list of portfolios

In [0]:
#use pandas to make it into a datafrmae
portfolios = pd.DataFrame(stuff_people_do, columns = ['f_id', 'gig_portfolio'])
#Calculate the most common gig_activity
portfolios['max'] = portfolios['gig_portfolio'].map(lambda t: max(t))

Feature extraction from lists and dictionaries
A big part of Data Science is feature extraction – producing data where is no "traditional" data. Much more on tha tin M2 and M3

But for now: How can we produce tabular data from lists of jobs?



In [0]:
list_of_lists = [['cat','dog','dog','horse','tree'], ['cat','cat','dog'],['elephant','cat','horse', 'horse'] ]

In [0]:
# Packages for feature extraction
from collections import Counter #helper package for counting stuff
from sklearn.feature_extraction import DictVectorizer

In [0]:
v = DictVectorizer(sparse=False)

v.fit_transform([{'cat':2, 'dog':1},  # ['cat','cat','dog']
                 {'cat':2, 'elephant':5}
                 ])

In [0]:
# This is where Counter comes in
Counter(['cat','cat','dog'])

In [0]:
dicts = []
for i in list_of_lists:
  dicts.append(dict(Counter(i)))

# Let's check:

print(dicts)

In [0]:
array = v.fit_transform(dicts)
print(array)

### Back to our job-portfolios

Let's apply this feature-extraction approach:



In [0]:
dicts = []
for i in portfolios['gig_portfolio']:
  dicts.append(dict(Counter(i)))

portfolio_matrix = v.fit_transform(dicts)

In [0]:
portfolio_matrix.shape

In [0]:
portfolio_matrix

Why should I trust this? Can we check if all the transformations didn't mess up things?

In [0]:
# portfolio of worker 0
portfolio_matrix[1]

In [0]:
# what's the taks sub-category of index 3?
v.feature_names_[1]

In [0]:
# How many times did the worker perform this gig?
data[data.f_id == 1].sub_category

Now we sucessfully created "Bag-of-Gigs" representations of individual workers' portfolios
The matrix is however very sparse, as we can see (many zeros and  only some non-0 values) as one would expect that.
PCA is not a good choice here. Instead we will use Non-negative Matrix Factorization (NMF), which has the tendency to very well "squash" the data into interpretable latent themes:

In Natural Language Processing, such themes are called topics and what we are going to do now is referred to as Vector Space Modelling or Topic Modelling

The coolest thing about that identified components are very interopretable. This is because the model is build on assumptions about the world related to co-occurence.

In [0]:
# Let's try to bring it all the way down to 5 dimensions

from sklearn.decomposition import NMF
model = NMF(n_components=5)
portfolio_matrix_reduced = model.fit_transform(portfolio_matrix)

In [0]:
# what are these components?
model.components_.shape

In [0]:
# Make a dataframe
components_df = pd.DataFrame(model.components_, columns=list(v.feature_names_))

In [0]:
components_df

In [0]:
# Select a component
component = components_df.iloc[0,:]

# Print result of nlargest
print(component.nlargest())

In [0]:
# Import clustering and dimensionality reduction
# HDBSCAN won't work with numpy < 1.16

import hdbscan
import umap

In [0]:
# Let's try not to overdo things and maybe keep it at 20 components

model = NMF(n_components=20)

portfolio_matrix_reduced = model.fit_transform(portfolio_matrix)

In [0]:
# Note that the standard setting of UMAP will produce 2 dimensions
embedding = umap.UMAP(n_neighbors=15, metric='cosine').fit_transform(portfolio_matrix_reduced)

You can read more about


- UMAP here: https://umap-learn.readthedocs.io/en/latest/
- HDBSCAN here: https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html

HDBSCAN is a recent hi-performance density based clustering approach


In [0]:
# Now, we will feed the 2-dimensional representation into HDBSCAN
# Warning can be ignored for now

clusterer = hdbscan.HDBSCAN(min_cluster_size=50, 
                            min_samples=50, 
                            leaf_size=40, 
                            #core_dist_n_jobs=16, 
                            prediction_data=True)
clusterer.fit(embedding)

In [0]:
pal = sns.color_palette("Paired", n_colors = len(set(clusterer.labels_)))[1:]
plt.figure(figsize=(12,12))
plt.rcParams.update({'font.size': 15})
clusterer.condensed_tree_.plot(select_clusters=True,
                               selection_palette=pal, label_clusters=True)

In [0]:
# Scatterplor of the UMAP embeddings with cluster-coloring
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(10,10))
g = sns.scatterplot(*embedding.T, 
                hue=clusterer.labels_, 
                legend='full',
                palette = 'Paired')
legend = g.get_legend()

### Let's quickly explore the clusters.
Here we only look at the most common activities in the cluster. But certainly one could compare the identified communities on other dimensions.

In [0]:
# Write community number into the initial dataset
portfolios['cluster'] = clusterer.labels_

In [0]:
# helper tool for iteration, combinations etc https://docs.python.org/2/library/itertools.html
import itertools

In [0]:
# Link up the job portfolios of freelancer in cluster and count up how many times tasks have been performed.
counter = Counter(list(itertools.chain(*portfolios[portfolios.cluster == 0]['gig_portfolio'])))

In [0]:
# Whow most common tasks
counter.most_common(20)